In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import pandas as pd

# # Define the path to your dataset CSV file
# csv_file_path = '/content/drive/MyDrive/Colab Notebooks/Leaf Disease/Train_tomato.csv'

# # Read the CSV file into a Pandas DataFrame
# df = pd.read_csv(csv_file_path)

# # Check the structure of the DataFrame
# print(df.head())


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
#from tensorflow.keras.callbacks import ReduceLROnPlateaulist
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D

In [ ]:
# # Split the data into training and validation sets (70% train, 30% validation)
# train_df, validation_df = train_test_split(df, test_size=0.3, random_state=42)

# # Check the number of samples in each set
# print("Train set size:", len(train_df))
# print("Validation set size:", len(validation_df))

# # Assuming your CSV file has columns 'image_path' and 'label'
# train_image_paths = train_df['Image labels'].tolist()
# train_labels = train_df['Labels'].tolist()

# validation_image_paths = validation_df['Image labels'].tolist()
# validation_labels = validation_df['Labels'].tolist()

In [ ]:
# Load the pre-trained VGG16 model without the top (fully connected) layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the weights of the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False


# Add BatchNormalization after each convolutional layer
x = base_model.output
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)

# Add custom layers for your specific classification task
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(10, activation='softmax')(x)  # 10 output classes

# Create the custom model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])





58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
# Data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load your dataset using the ImageDataGenerator
train_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/Tomato_train_new',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/Leaf Disease/Tomato_test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 8061 images belonging to 10 classes.
Found 2295 images belonging to 10 classes.


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define a callback to save the best model
checkpoint = ModelCheckpoint("/content/drive/MyDrive/Colab Notebooks/Leaf Disease/2_2_VGG16_new.h5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# Train the model with the callback
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=40,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[checkpoint]
)

Epoch 1/40
252/252 [==============================] - ETA: 0s - loss: 1.9768 - accuracy: 0.2893
Epoch 1: val_loss improved from inf to 1.85766, saving model to /content/drive/MyDrive/Colab Notebooks/Leaf Disease/2_2_VGG16_new.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


252/252 [==============================] - 2206s 9s/step - loss: 1.9768 - accuracy: 0.2893 - val_loss: 1.8577 - val_accuracy: 0.3495
Epoch 2/40
252/252 [==============================] - ETA: 0s - loss: 1.6488 - accuracy: 0.4217
Epoch 2: val_loss improved from 1.85766 to 1.66270, saving model to /content/drive/MyDrive/Colab Notebooks/Leaf Disease/2_2_VGG16_new.h5
252/252 [==============================] - 166s 658ms/step - loss: 1.6488 - accuracy: 0.4217 - val_loss: 1.6627 - val_accuracy: 0.4240
Epoch 3/40
252/252 [==============================] - ETA: 0s - loss: 1.5130 - accuracy: 0.4865
Epoch 3: val_loss improved from 1.66270 to 1.59709, saving model to /content/drive/MyDrive/Colab Notebooks/Leaf Disease/2_2_VGG16_new.h5
252/252 [==============================] - 157s 624ms/step - loss: 1.5130 - accuracy: 0.4865 - val_loss: 1.5971 - val_accuracy: 0.4519
Epoch 4/40
252/252 [==============================] - ETA: 0s - loss: 1.4324 - accuracy: 0.4987
Epoch 4: val_loss improved from 1.5

In [ ]:
# Save the trained model
model.save('/content/drive/MyDrive/Colab Notebooks/Leaf Disease/Models/vgg16_transfer_learning_model_3.h5')

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/Leaf Disease/Models/vgg16_transfer_learning_model_1.h5')

In [ ]:
import os

In [ ]:
os.path.getsize('/content/drive/MyDrive/Leaf Disease/Models/vgg16_transfer_learning_model_1.h5')

136051528

In [ ]:
model_tflite = 'vgg16_transfer_learning_model_3.tflite'
tf_lite_convertor = tf.lite.TFLiteConverter.from_keras_model(model)
tf_lite_model = tf_lite_convertor.convert()
open(model_tflite, "wb").write(tf_lite_model)

84569652

In [ ]:
1 - 0.6213235294117647

0.3786764705882353

In [ ]:
interpreter = tf.lite.Interpreter(model_path = model_tflite)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input shape",input_details[0]['shape'])
print("Input Type",input_details[0]['dtype'])
print("Output shape",output_details[0]['shape'])
print("Output Type",output_details[0]['dtype'])

Input shape [  1 224 224   3]
Input Type <class 'numpy.float32'>
Output shape [ 1 10]
Output Type <class 'numpy.float32'>


In [ ]:
tf_lite_convertor.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = tf_lite_convertor.convert()
quantized_model = 'quantized_model.tflite'
open(quantized_model,"wb").write(tflite_model)

21214120

In [ ]:
tf_lite_convertor.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = tf_lite_convertor.convert()
quantized_model = 'quantized_model1.tflite'
open(quantized_model,"wb").write(tflite_model)

21214120

In [ ]:
1 - 2121/13605

0.8441014332965822

In [ ]:
os.path.getsize(model_tflite)

84569652